In [ ]:
import gradio as gr
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import os

# Load the trained model
model = tf.keras.models.load_model('transfer_model.h5')

# Define class labels for diabetic retinopathy
class_labels = ['No DR', 'Mild DR', 'Moderate DR', 'Severe DR', 'Proliferative DR']

# Function to preprocess the uploaded image
def preprocess_image(image):
    # Resize image to match model input (assuming 224x224 based on common ResNet50 input)
    image = image.resize((224, 224))
    # Convert to array and normalize
    image_array = img_to_array(image)
    image_array = image_array / 255.0
    # Expand dimensions to match model input shape
    image_array = np.expand_dims(image_array, axis=0)
    return image_array

# Function to predict diabetic retinopathy from an image
def predict_diabetes(image):
    try:
        # Preprocess the image
        processed_image = preprocess_image(image)
        
        # Make prediction
        predictions = model.predict(processed_image)
        predicted_class = np.argmax(predictions[0])
        confidence = float(np.max(predictions[0])) * 100
        
        # Get prediction label
        prediction_label = class_labels[predicted_class]
        
        # Prepare confidence scores for all classes
        confidence_scores = {class_labels[i]: float(predictions[0][i]) * 100 for i in range(len(class_labels))}
        
        # Note: Model accuracy is from your classification report (weighted avg)
        model_accuracy = 66  # From your classification report: accuracy 0.66
        
        # Prepare output
        output = (
            f"**Prediction**: {prediction_label}\n"
            f"**Confidence**: {confidence:.2f}%\n"
            f"**Model Accuracy**: {model_accuracy}%\n\n"
            f"**Confidence Scores for All Classes**:\n"
        )
        for label, score in confidence_scores.items():
            output += f"- {label}: {score:.2f}%\n"
        
        return output
    except Exception as e:
        return f"Error processing image: {str(e)}"

# Create Gradio interface
iface = gr.Interface(
    fn=predict_diabetes,
    inputs=gr.Image(type="pil", label="Upload Retina Image"),
    outputs=gr.Textbox(label="Prediction Results"),
    title="Diabetic Retinopathy Prediction",
    description="Upload a retina image to predict the level of diabetic retinopathy. The model provides a prediction, confidence score, and overall model accuracy.",
    allow_flagging="never"
)

# Launch the interface
iface.launch()
